In [ ]:
from typing import List

import pandas as pd
import math
import networkx
import matplotlib.pyplot as plt
import requests
import json

In [ ]:
def get_interaction_network(species):
    string_api_url = "https://version-11-5.string-db.org/api"
    output_format = "tsv-no-header"
    method='network'
    
    genes = pd.read_csv(f'data/{species}.tsv', sep='\t')['Gene names  (primary )'].tolist()
    # some genes are empty (rendered as proper float nan's, filter out)
    
    genes = list(filter(lambda x: type(x) != float, genes ))
    
    request_url = "/".join([string_api_url, output_format, method])

    params = {

        "identifiers" : "%0d".join(genes),
        "species" : species, 
        "limit" : 20,
        "required_score": 100,

    }

    response = requests.post(request_url, data=params)
    
    interactions = []
    
    
    for line in response.text.strip().split("\n"):

        l = line.strip().split("\t")
        query_ensp = l[0]
        query_name = l[2]
        partner_ensp = l[1]
        partner_name = l[3]
        combined_score = l[5]
        
        interactions.append([query_ensp, query_name,partner_ensp, partner_name, combined_score ])
    int_df = pd.DataFrame(interactions, columns=['query_ensp', 'query_name', 'partner_ensp', 'partner_name', 'combined_score'])    
    return genes, int_df

In [ ]:
# Some GO terms that we want to filter out from functional enrichment 

obvious_go_terms = ['GO:0046677', # response to antibiotics
                    'GO:0042221', # response to checmical
                    'GO:0050896'] # response to stimulus

In [ ]:
def get_functional_enrichment(genes, resistant_genes, species):
    string_api_url = "https://version-11-5.string-db.org/api"
    output_format = "json"
    method = "enrichment"


    request_url = "/".join([string_api_url, output_format, method])

    params = {

        "identifiers" : "%0d".join(genes), # your protein
        "species" : species, # species NCBI identifier 
    }


    response = requests.post(request_url, data=params)

    data = json.loads(response.text)
    df =  pd.DataFrame(data)
    df = df[~df['term'].isin(obvious_go_terms)]
    df['neighbourhoodGenes'] = df['inputGenes'].apply(lambda x: list(set(x).difference(resistant_genes)))
    df = df[df['category'] == 'Process'] 
    
    
    return df

In [ ]:
def draw_with_resistance_map(resistance_genes, G):
    color_map = []
    for node in G:
        if node in resistance_genes:
            color_map.append('red')
        else: 
            color_map.append('blue')
    fig = plt.figure(1, figsize=(12, 8), dpi=300)
    networkx.draw(G, with_labels=True, node_size=1000, node_color=color_map)

In [ ]:
def get_all_interacting_nodes(df: pd.DataFrame) -> List[str]:
    return pd.concat([df['query_name'] ,df['partner_name']]).unique().tolist()

# Gram-positive

### Enterococcus faecalis (strain ATCC 700802 / V583)

In [ ]:
226185
resistance_genes, df = get_interaction_network(226185)
# dfw
G = networkx.from_pandas_edgelist(df, 'query_name', 'partner_name', edge_attr=True)
draw_with_resistance_map(resistance_genes, G)
int_genes = get_all_interacting_nodes(df)
get_functional_enrichment(int_genes, resistance_genes, 226185)

### Staphylococcus aureus (strain NCTC 8325 / PS 47)

In [ ]:
226185
resistance_genes, df = get_interaction_network(93061)
# dfw
G = networkx.from_pandas_edgelist(df, 'query_name', 'partner_name', edge_attr=True)
draw_with_resistance_map(resistance_genes, G)
int_genes = get_all_interacting_nodes(df)
get_functional_enrichment(int_genes, resistance_genes, 1280)

In [ ]:
resistance_genes, df = get_interaction_network(93061)
# dfw
G = networkx.from_pandas_edgelist(df, 'query_name', 'partner_name', edge_attr=True)
draw_with_resistance_map(resistance_genes, G)
int_genes = get_all_interacting_nodes(df)
get_functional_enrichment(int_genes, 1280)

# Gram negative

In [ ]:
resistance_genes, df = get_interaction_network(272634)
# dfw
G = networkx.from_pandas_edgelist(df, 'query_name', 'partner_name', edge_attr=True)
draw_with_resistance_map(resistance_genes, G)
int_genes = get_all_interacting_nodes(df)
get_functional_enrichment(int_genes, resistance_genes, 722438)

In [ ]:
genes, df = get_interaction_network(272634 )
# dfw
G = networkx.from_pandas_edgelist(df, 'query_name', 'partner_name', edge_attr=True)

draw_with_resistance_map(genes, G)

In [ ]:
genes, df = get_interaction_network(43263 )
# dfw
G = networkx.from_pandas_edgelist(df, 'query_name', 'partner_name', edge_attr=True)

draw_with_resistance_map(genes, G)